In [16]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
from bs4 import BeautifulSoup
import json
import pyarabic.araby as araby
from tashaphyne.stemming import ArabicLightStemmer
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import qalsadi.lemmatizer as lem



In [ ]:


# Configure options for Chrome
options = Options()
options.add_argument('--headless')  # Run in headless mode for non-GUI environments
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

# Set up the Chrome WebDriver
driver = webdriver.Chrome()


# Open the website
driver.get('https://www.aljazeera.net/where/arab/palestine/')

links = set()
for i in range(30):
    # Wait until the links are present
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'u-clickable-card__link'))
    )

    # Find all link elements
    link_elements = driver.find_elements(By.CLASS_NAME, 'u-clickable-card__link')
    for element in link_elements:
        links.add(element.get_attribute('href'))

    # Try to find and click the "Show More" button
    show_more_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[3]/div/div[3]/div/div[1]/section/button'))
    )
    driver.execute_script('arguments[0].scrollIntoView();', show_more_button)
    show_more_button.click()
    time.sleep(3)  # Wait for new content to load

# Output all links
for link in links:
    print(link)

driver.quit()


In [12]:
len(links)

304

In [ ]:

para=[]
for url in links:
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    para_div = soup.find('div', {'class': 'wysiwyg wysiwyg--all-content css-1vkfgk0'})
    if para_div:
        print("URL:", url)
         
        
        # Initialize a variable to store all paragraphs
        all_paragraphs = ""

        # Extract paragraphs
        for item in para_div.contents:
            if item.name == 'p':
                all_paragraphs += item.text.strip() + "  "

        print("Paragraphs:", all_paragraphs)
        para.append(all_paragraphs)
        print("\n")  # Add a new line for separation between articles
        # Insert data into MongoDB
        article_data = {
            "url": url,
            "paragraphs": all_paragraphs
        }


In [21]:
# Step 3: Open (or create) the 'para.json' file in write mode
with open('para.json', 'w') as json_file:
    # Step 4: Use json.dump() to write the list to the file
    json.dump(para, json_file)

print("List has been written to para.json")


List has been written to para.json


In [33]:
df

,0
0,بثت كتائب عز الدين القسام -الجناح العسكري لحرك...
1,يعيش المصابون بأمراض مزمنة في قطاع غزة، الحرب ...
2,أفادت مصادر إسرائيلية بمقتل وجرح 20 جنديا اليو...
3,عرضت كتائب القسام -الجناح العسكري لحركة المقاو...
4,في اليوم الـ220 للعدوان الإسرائيلي على غزة، تص...
...,...
253,يرى الخبير في الشؤون الإستراتيجية والعسكرية ال...
254,أعلنت المقاومة الفلسطينية مساء أمس الخميس استه...
255,تواصل الحراك الطلابي المناهض للحرب الإسرائيلية...
256,حذرت الجبهة الشعبية لتحرير فلسطين، ولجان المقا...


In [ ]:
# Configure options for Chrome
options = Options()
options.add_argument('--headless')  # Run in headless mode for non-GUI environments
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

# Set up the Chrome WebDriver
driver = webdriver.Chrome()


# Open the website
driver.get('https://www.aljazeera.net/')

links = set()
for i in range(1):
    # Wait until the links are present
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'u-clickable-card__link'))
    )

    # Find all link elements
    link_elements = driver.find_elements(By.CLASS_NAME, 'u-clickable-card__link')
    for element in link_elements:
        links.add(element.get_attribute('href'))

    # # Try to find and click the "Show More" button
    # show_more_button = WebDriverWait(driver, 10).until(
    #     EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[3]/div/div[3]/div/div[1]/section/button'))
    # )
    # driver.execute_script('arguments[0].scrollIntoView();', show_more_button)
    # show_more_button.click()
    # time.sleep(3)  # Wait for new content to load

# Output all links
for link in links:
    print(link)

driver.quit()   

In [ ]:
para=[]
for url in links:
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    para_div = soup.find('div', {'class': 'wysiwyg wysiwyg--all-content css-1vkfgk0'})
    if para_div:
        print("URL:", url)
         
        
        # Initialize a variable to store all paragraphs
        all_paragraphs = ""

        # Extract paragraphs
        for item in para_div.contents:
            if item.name == 'p':
                all_paragraphs += item.text.strip() + "  "

        print("Paragraphs:", all_paragraphs)
        para.append(all_paragraphs)
        print("\n")  # Add a new line for separation between articles
        # Insert data into MongoDB
        article_data = {
            "url": url,
            "paragraphs": all_paragraphs
        }

In [ ]:
# Create a list to hold the dictionaries
data_list = []

# Iterate over each item in the 'para' list
for content in para:
    # Create a dictionary with 'content' and 'title'
    data = {
        "title": "None",
        "content": content,  
    }
    
    # Add the dictionary to the list
    data_list.append(data)

# Write the list of dictionaries to a single JSON file
with open('para2.json', 'w', encoding='utf-8') as json_file:
    json.dump(data_list, json_file, ensure_ascii=False, indent=4)

print("All items have been written to 'para2.json'.")

In [6]:
# Read the JSON file with lines=True
df = pd.read_json("filtered_&_scored_data.json", lines=True)

# Drop the column named 'B'
if 'B' in df.columns:
    df = df.drop(columns=['B'])

# Save the updated DataFrame back to a JSON file
df.to_json('updated_para.json', orient='records', lines=True, force_ascii=False, indent=4)

# Print the updated DataFrame
print(df)


                                               content    rating
0    بثت كتائب عز الدين القسام -الجناح العسكري لحرك...  6.305360
1    يعيش المصابون بأمراض مزمنة في قطاع غزة، الحرب ...  9.647305
2    أفادت مصادر إسرائيلية بمقتل وجرح 20 جنديا اليو...  7.359289
3    عرضت كتائب القسام -الجناح العسكري لحركة المقاو...  9.395899
4    في اليوم الـ220 للعدوان الإسرائيلي على غزة، تص...  9.478555
..                                                 ...       ...
206  طهران- في تصريحات رسمية يمكن اعتبار أنها الأول...  4.024692
207  حاول الغرب، على مدى السنوات القليلة الماضية، ك...  1.148186
208  يحذر خبراء في جنوب أفريقيا من عودة الرئيس السا...  1.615157
209  في سياق إقليمي معقد تمر به منطقة أفريقيا جنوب ...  3.324926
210  قرر يوفنتوس الإيطالي الطعن في حكم كريستيانو رو...  0.855067

[211 rows x 2 columns]


In [8]:
df['content']

0      بثت كتائب عز الدين القسام -الجناح العسكري لحرك...
1      يعيش المصابون بأمراض مزمنة في قطاع غزة، الحرب ...
2      أفادت مصادر إسرائيلية بمقتل وجرح 20 جنديا اليو...
3      عرضت كتائب القسام -الجناح العسكري لحركة المقاو...
4      في اليوم الـ220 للعدوان الإسرائيلي على غزة، تص...
                             ...                        
206    طهران- في تصريحات رسمية يمكن اعتبار أنها الأول...
207    حاول الغرب، على مدى السنوات القليلة الماضية، ك...
208    يحذر خبراء في جنوب أفريقيا من عودة الرئيس السا...
209    في سياق إقليمي معقد تمر به منطقة أفريقيا جنوب ...
210    قرر يوفنتوس الإيطالي الطعن في حكم كريستيانو رو...
Name: content, Length: 211, dtype: object

In [17]:
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",'\n', '\t','"','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    # remove HTML TAG
    html = re.compile('[<،,()"".#*?>]')
    text = html.sub(r'',text)
    
    # Remove urls:
    url = re.compile('https?://\S+|www\.S+')
    text = url.sub(r'',text)
    
    # Remove email id:
    email = re.compile('[A-Za-z0-2]+@[\w]+.[\w]+')
    text = email.sub(r'',text)
    
    # Remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    # Remove longation "EX : سلاممممم = سلامم"
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    # remove any leading and trailing whitespace characters    
    text = text.strip()
    return text

data=df['content'] 
# para = clean_str(data)


TypeError: expected string or bytes-like object, got 'Series'

In [19]:
data=df['content'] 
data

0      بثت كتائب عز الدين القسام -الجناح العسكري لحرك...
1      يعيش المصابون بأمراض مزمنة في قطاع غزة، الحرب ...
2      أفادت مصادر إسرائيلية بمقتل وجرح 20 جنديا اليو...
3      عرضت كتائب القسام -الجناح العسكري لحركة المقاو...
4      في اليوم الـ220 للعدوان الإسرائيلي على غزة، تص...
                             ...                        
206    طهران- في تصريحات رسمية يمكن اعتبار أنها الأول...
207    حاول الغرب، على مدى السنوات القليلة الماضية، ك...
208    يحذر خبراء في جنوب أفريقيا من عودة الرئيس السا...
209    في سياق إقليمي معقد تمر به منطقة أفريقيا جنوب ...
210    قرر يوفنتوس الإيطالي الطعن في حكم كريستيانو رو...
Name: content, Length: 211, dtype: object